# Network State Index -- API demo

see: https://github.com/yzerlaut/Network_State_Index

We demonstrate the use of the API on the following publicly available dataset:

## the "Visual Coding – Neuropixels" dataset from the Allen Observatory

All details about this dataset and instructions for analysis are available at:

https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html

## Dataset download

I adapted the download instructions and made a [custom script](https://github.com/yzerlaut/Network_State_Index/blob/main/demo/download_Allen_Visual-Coding_dataset.py) to download exclusively the part of the dataset of interest here (i.e. V1 probes, ~20GB of LFP data).
You can run the script as:
```
python demo/download_Allen_Visual-Coding_dataset.py
```

In [1]:
# some general python / scientific-python modules
import os, shutil
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
#plt.style.use('seaborn')
plt.style.use('ggplot')

# the Network State Index API, see: https://github.com/yzerlaut/Network_State_Index
# install it with: "pip install git+https://github.com/yzerlaut/Network_State_Index"
import nsi


V1_Allen_params = {'pLFP_band':[40,140], # Hz -- paper's values 
                   'delta_band':[4,8], # Hz -- the delta peaks at 6hz, not 3hz..
                   'Tsmoothing':30e-3,  # s -- slightly lower smoothing because the 42ms smoothing of S1 smooth out a bit too much the delta in V1
                   'alpha_LFP':3., # evaluated below
                   'alpha_rate':2} # the rate is a virtually noiseless signal (it defines Network States), so no need to be >2 (see paper's derivation)

### We restrict the analysis to:

- wild type / wild type strain
- male
- sessions with probes in V1 ("VISp")
- "functional_connectivity" dataset because it has 30min of spontaneous activity (we pick 20min in the center of that period)

In [2]:
# Download and load the data with the "allensdk" API
# get the "allensdk" api with: "pip install allensdk"
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

# now let's define a cache repository for the data: by default ~/Downloads/ecephys_cache_dir
# insure that you have a "Downloads" repository in your home directory (/!\ non-english systems) or update below
data_directory = os.path.join(os.path.expanduser('~'), 'Downloads', 'ecephys_cache_dir')
manifest_path = os.path.join(data_directory, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
all_sessions = cache.get_session_table() # get all sessions

# let's filter the sessions according to the above criteria
sessions = all_sessions[(all_sessions.sex == 'M') & \
                        (all_sessions.full_genotype.str.find('wt/wt') > -1) & \
                        #(all_sessions.session_type == 'brain_observatory_1.1') & \
                        (all_sessions.session_type == 'functional_connectivity') & \
                        (['VISp' in acronyms for acronyms in all_sessions.ecephys_structure_acronyms])]
print(30*'--'+'\n--> Number of sessions with the desired characteristics: ' + str(len(sessions))+'\n'+30*'--')
# sessions.head() # uncomment to see how they look

------------------------------------------------------------
--> Number of sessions with the desired characteristics: 11
------------------------------------------------------------


## Loading, formatting and preprocessing the data

In [56]:
import time

   
class Data:
    """
    an object to load, format and process the data
    we use the Allen SDK to fetch the V1 channels
    
    we format things so that data are accessible as: "data.QUANTITY" with time sampling data.t_QUANTITY (e.g. "data.pLFP", "data.t_pLFP")
    """
    
    def __init__(self, 
                 session_index=0,
                 reduced=False,
                 demo=False, demo_filename='allen_demo_sample.npy',
                 t0=115*60, 
                 duration=20*60, # 20 min by default
                 #init=['pop_act', 'pLFP', 'NSI'], # for a full init
                 init = []):
        """
        loading data:
        - the Allen NWB files are loaded by default, select the session according to the index of the "sessions" above
        - for initial troubleshooting, there is a "demo" option that loads a lightweight data sample provided in the repo
        - for faster analysis alter (to loop over the data), we can load the "reduced" version of the data of interest (generated below)


        can load a subset using the "t0" and "duration" args
        """
        
        if demo:
            # -------- using the stored demo data if "demo" mode ----- #
            DEMO = np.load(demo_filename, allow_pickle=True).item()
            for key in DEMO:
                setattr(self, key, DEMO[key]) # sets LFP
            self.t_LFP = np.arange(len(self.LFP))/self.lfp_sampling_rate+self.t0
        elif reduced:
            rdata = np.load('reduced_data/Allen_FC_session%i.npy' % (session_index+1), allow_pickle=True).item()
            for key in rdata:
                setattr(self, key, rdata[key]) # sets LFP, pLFP, pop_act, running_speed
            for key in ['LFP', 'pLFP', 'pop_act']:
                setattr(self, 't_%s'%key, np.arange(len(getattr(self,key)))/getattr(self,'%s_sampling_rate'%key)+self.t0)
        else:
            # -------------------------------------------------------- # 
            # -- using the Allen SDK to retrieve and cache the data -- #
            # -------------------------------------------------------- # 

            print('loading session #%i [...]' % (1+session_index))
            tic = time.time()
            # we load a single session
            session = cache.get_session_data(sessions.index.values[session_index])

            # use the running timestamps to set start and duration in the data object
            self.t0 = np.max([t0, session.running_speed.start_time.values[0]])
            self.duration = np.min([duration, session.running_speed.end_time.values[-1]-self.t0])

            # let's fetch the running speed
            cond = (session.running_speed.end_time.values>self.t0) &\
                (session.running_speed.start_time.values<(self.t0+self.duration))
            self.t_running_speed = .5*(session.running_speed.start_time.values[cond]+\
                                       session.running_speed.end_time.values[cond])
            self.running_speed = session.running_speed.velocity[cond]

            # let's fetch the isolated single units in V1
            V1_units = session.units[session.units.ecephys_structure_acronym == 'VISp'] # V1==VISp
            self.V1_RASTER = []
            for i in V1_units.index:
                cond = (session.spike_times[i]>=self.t0) & (session.spike_times[i]<(self.t0+self.duration))
                self.V1_RASTER.append(session.spike_times[i][cond])

            # let's fetch the V1 probe --> always on "probeC"
            probe_id = session.probes[session.probes.description == 'probeC'].index.values[0]

            # -- let's fetch the lfp data for that probe and that session --
            # let's fetch the all the channels falling into V1 domain
            self.V1_channel_ids = session.channels[(session.channels.probe_id == probe_id) & \
                          (session.channels.ecephys_structure_acronym.isin(['VISp']))].index.values

            # limit LFP to desired times and channels
            # N.B. "get_lfp" returns a subset of all channels above
            self.lfp_slice_V1 = session.get_lfp(probe_id).sel(time=slice(self.t0,
                                                                         self.t0+self.duration),
                                                              channel=slice(np.min(self.V1_channel_ids), 
                                                                            np.max(self.V1_channel_ids)))
            self.Nchannels_V1 = len(self.lfp_slice_V1.channel) # store number of channels with LFP in V1
            self.lfp_sampling_rate = session.probes.lfp_sampling_rate[probe_id] # keeping track of sampling rate
            print('data successfully loaded in %.1fs' % (time.time()-tic))
              
        for key in init:
            getattr(self, 'compute_%s' % key)()
            

    def update_t0_duration(self, t0, duration):
        t0 = t0 if (t0 is not None) else self.t0
        duration = duration if (duration is not None) else self.duration
        return t0, duration
    
        
    def compute_pop_act(self, 
                        pop_act_bin=5e-3,
                        pop_act_smoothing=V1_Allen_params['Tsmoothing']):
        """
        we bin spikes to compute population activity
        """
        print(' - computing pop_act from raster [...]') 
        t_pop_act = self.t0+np.arange(int(self.duration/pop_act_bin)+1)*pop_act_bin
        pop_act = np.zeros(len(t_pop_act)-1)

        for i, spikes in enumerate(self.V1_RASTER):
            pop_act += np.histogram(spikes, bins=t_pop_act)[0]
        pop_act /= (len(self.V1_RASTER)*pop_act_bin)

        self.t_pop_act = .5*(t_pop_act[1:]+t_pop_act[:-1])
        self.pop_act = nsi.gaussian_filter1d(pop_act, 
                                             int(pop_act_smoothing/pop_act_bin)) # filter from scipy
        self.pop_act_sampling_rate = 1./pop_act_bin
        print(' - - > done !') 
        
        
    def compute_NSI(self, quantity='pLFP',
                    low_freqs = np.linspace(*V1_Allen_params['delta_band'], 5),
                    p0_percentile=1.,
                    alpha=2.87,
                    T_sliding_mean=500e-3,
                    with_subquantities=True,
                    verbose=True):
        """
        ------------------------------
            HERE we use the NSI API
        ------------------------------
        """
        if verbose:
            print(' - computing NSI for "%s" [...]' % quantity) 
        setattr(self, '%s_0' % quantity, np.percentile(getattr(self, quantity), p0_percentile/100.))
        
        lfe, sm, NSI = nsi.compute_NSI(getattr(self, quantity),
                                       getattr(self, '%s_sampling_rate' % quantity),
                                       low_freqs = low_freqs,
                                       p0=getattr(self, '%s_0' % quantity),
                                       alpha=alpha,
                                       T_sliding_mean=T_sliding_mean, 
                                       with_subquantities=True) # we fetch also the NSI subquantities (low-freq env and sliding mean), set below !
        setattr(self, '%s_low_freq_env' % quantity, lfe)
        setattr(self, '%s_sliding_mean' % quantity, sm)
        setattr(self, '%s_NSI' % quantity, NSI)
        if verbose:
            print(' - - > done !') 
        
    def validate_NSI(self, quantity='pLFP',
                     Tstate=200e-3,
                     var_tolerance_threshold=None,
                     verbose=True):
        """
        ------------------------------
            HERE we use the NSI API
        ------------------------------
        """
        if verbose:
            print(' - validating NSI for "%s" [...]' % quantity) 
        
        if var_tolerance_threshold is None:
            # by default the ~noise level evaluated as the first percentile
            var_tolerance_threshold = getattr(self, '%s_0' % quantity)
 
        vNSI = nsi.validate_NSI(getattr(self, 't_%s' % quantity),
                                getattr(self, '%s_NSI' % quantity),
                                Tstate=Tstate,
                                var_tolerance_threshold=var_tolerance_threshold)
    
        setattr(self, 'i_%s_vNSI' % quantity, vNSI)
        setattr(self, 't_%s_vNSI' % quantity, getattr(self, 't_%s' % quantity)[vNSI])
        setattr(self, '%s_vNSI' % quantity, getattr(self, '%s_NSI' % quantity)[vNSI])
        if verbose:
            print(' - - > done !')
        
    def plot(self, quantity, 
             t0=None, duration=None,
             ax=None, label='',
             subsampling=1,
             color='k', ms=0, lw=1, alpha=1):
        """
        quantity as a string (e.g. "pLFP" or "running_speed")
        """
        
        t0, duration = self.update_t0_duration(t0, duration)
        
        try:
            if ax is None:
                fig, ax =plt.subplots(1, figsize=(8,3))
            else:
                fig = None
            t = getattr(self, 't_'+quantity.replace('_NSI','').replace('_low_freq_env','').replace('_sliding_mean',''))
            signal = getattr(self, quantity)
            cond = (t>t0) & (t<(t0+duration))
            ax.plot(t[cond][::subsampling], signal[cond][::subsampling], color=color, lw=lw, ms=ms, marker='o', alpha=alpha)
            ax.set_ylabel(label)
            return fig, ax
        except BaseException as be:
            print(be)
            print('%s not a recognized attribute to plot' % quantity)
            return None, None
        
from scipy.interpolate import interp1d
# a tool very useful to 
def resample_trace(old_t, old_data, new_t):
    func = interp1d(old_t, old_data, kind='nearest', fill_value="extrapolate")
    return func(new_t)


## Channel selection in LFP recordings with Neuropixel probes

We benefit from many channels in the area of interest (~20 in V1). How to deal with this ? 

--> simple solution: we pick just one channel, the one that has the highest delta envelope in the pLFP. This sounds like a good guess for a channel with good physiological signal.

In [4]:
def find_channel_with_highest_delta(data,
                                    t0=None, duration=None,
                                    pLFP_band=V1_Allen_params['pLFP_band'],
                                    delta_band=V1_Allen_params['delta_band'],
                                    pLFP_subsampling=5,
                                    return_all=False):
    """
    A function to pick the 
    """
    channel_mean_delta, channel_id, final_pLFP, final_LFP = 0, None, None, None

    
    ALL = None
    if return_all:
        ALL = {'LFP':[], 'pLFP':[], 't':data.lfp_slice_V1.time}

    for c in range(len(data.lfp_slice_V1.channel.values)):
        # first get the LFP
        LFP = 1e3*np.array(data.lfp_slice_V1.sel(channel=data.lfp_slice_V1.channel[c]))
        if return_all:
            ALL['LFP'].append(LFP)
        # then compute the pLFP            === USING the *nsi* API ===
        _, pLFP = nsi.compute_pLFP(LFP, data.lfp_sampling_rate,
                                   smoothing=V1_Allen_params['Tsmoothing'])
        if return_all:
            ALL['pLFP'].append(1e3*pLFP)
        # compute low freq envelope of pLFP (subsampled on need of full sampling)
        lf_env = nsi.compute_freq_envelope(1e3*pLFP[::pLFP_subsampling], 
                                           data.lfp_sampling_rate/pLFP_subsampling,
                                           np.linspace(delta_band[0], delta_band[1], 5))
        
        if np.mean(lf_env)>channel_mean_delta:
            channel_mean_delta = np.mean(lf_env)
            final_LFP = LFP
            final_pLFP = 1e3*pLFP
            channel_id = c
    return final_LFP, final_pLFP, channel_id, ALL


In [5]:
data = Data(session_index=0, 
            init=['pop_act'])

final_LFP, final_pLFP, channel_id, ALL = find_channel_with_highest_delta(data,
                                                                         return_all=True)

loading session #1 [...]
data successfully loaded in 34.8s
 - computing pop_act from raster [...]
 - - > done !


In [ ]:
t0, duration = data.t0+120, 5 # showing a sample with some nice sinal variations
fig, AX = plt.subplots(3, figsize=(9,6))
cmap = plt.cm.copper

cond = (ALL['t']>t0) & (ALL['t']<(t0+duration))
AX[1].plot(ALL['t'][cond], final_pLFP[cond], lw=3, color='darkgrey')
for i in range(len(ALL['LFP'])):
    AX[0].plot(ALL['t'][cond], ALL['LFP'][i][cond], lw=0.3, color=cmap(1-i/(data.Nchannels_V1-1)))
    AX[1].plot(ALL['t'][cond], ALL['pLFP'][i][cond], lw=0.3, color=cmap(1-i/(data.Nchannels_V1-1)))
        
data.plot('pop_act', ax=AX[2], t0=t0, duration=duration)
AX[2].set_ylabel('rate (Hz)  ')
AX[2].set_xlabel('time (s)  ')
AX[0].set_ylabel('LFP (mV)')
AX[1].set_ylabel('pLFP (uV)')
AX[1].annotate('selected channel ID: %i' % channel_id,
               (1,.9), xycoords='axes fraction', va='top', ha='right',color='darkgrey', weight='bold')
for i in range(data.Nchannels_V1):
    AX[0].annotate((i+1)*'      '+'                 %i' % i, (0,1), xycoords='axes fraction', va='top',
                color=cmap(1-i/(data.Nchannels_V1-1)))
AX[0].annotate('channel ID:\n(depth-ordered)', (0,1), xycoords='axes fraction', va='top');
plt.tight_layout()
fig.savefig('../doc/Neuropixels-channel-selection.png')

#### Now we loop over all sessions to get the reduced data with those properties

In [ ]:
# takes about 10 minutes on a good desktop computer

def save_reduced_data(data, session_index, LFP, pLFP, channel_id):
    new_data = {'t0':data.t0, 'duration':data.duration,
                'V1_RASTER':data.V1_RASTER,
                'pop_act':data.pop_act, 'pop_act_sampling_rate':data.pop_act_sampling_rate,
                'selected_channel_id':channel_id,
                'LFP':LFP, 'LFP_sampling_rate':data.lfp_sampling_rate,
                'pLFP':pLFP, 'pLFP_sampling_rate':data.lfp_sampling_rate,
                't_running_speed':data.t_running_speed, 'running_speed':data.running_speed,
               }
    np.save('reduced_data/Allen_FC_session%i.npy' % (session_index+1), new_data)

    
for session_index in range(len(sessions)):
    
    data = Data(session_index=session_index, reduced=False, demo=False,
                init=['pop_act'])
    print('looking for the best recording channel [...]')
    LFP, pLFP, channel_id, _ = find_channel_with_highest_delta(data)
    save_reduced_data(data, session_index, LFP, pLFP, channel_id)
    

## Characterizing the delta oscillation in V1

In [9]:
freqs = np.linspace(1, 10, 40)

envelope = np.zeros((len(freqs), len(sessions)))

for i in range(len(sessions)):
    data = Data(session_index=i, reduced=True)
    # wavelet transform using "NSI"
    envelope[:,i] = 1e3*np.abs(nsi.my_cwt(data.LFP, freqs, 
                                      1./data.LFP_sampling_rate)).mean(axis=1)


In [26]:
max_env_levels = [freqs[np.argmax(envelope[:,i])] for i in range(len(sessions))]
print('in V1, delta peak at: %.1f +/- %.1f Hz' % (np.mean(max_env_levels), np.std(max_env_levels)))

in V1, delta peak at: 5.7 +/- 0.6 Hz


In [11]:
# let's compare to S1, loading a reduced dataset per cell

if os.path.isfile('reduced_data/reduced_data_cell_1.npz'):
    # if you have the S1 data, let's do the same on S1
    envelope_S1 = np.zeros((len(freqs), 14))
    for i in np.arange(1, 15):
        data = dict(np.load('reduced_data/reduced_data_cell_'+str(i)+'.npz'))
        envelope_S1[:,i-1] = 1e3*np.abs(nsi.my_cwt(data['sbsmpl_Extra'], freqs, 
                                             data['sbsmpl_dt'])).mean(axis=1)
else:
    envelope_S1 = None

In [ ]:
fig, ax = plt.subplots(1, figsize=(2.3,2.3))

if envelope_S1 is not None:
    ax.plot(freqs, envelope_S1.mean(axis=1), color='darkgreen', lw=1)
    ax.fill_between(freqs, 
                     envelope_S1.mean(axis=1)-envelope_S1.std(axis=1),
                     envelope_S1.mean(axis=1)+envelope_S1.std(axis=1), color='darkgreen', alpha=0.2, lw=0)
    ax.annotate('S1           \n', (1,0.05), color='darkgreen', xycoords='axes fraction', ha='right')
    
ax.plot(freqs, envelope.mean(axis=1), color='darkblue')
ax.fill_between(freqs, 
                 envelope.mean(axis=1)-envelope.std(axis=1),
                 envelope.mean(axis=1)+envelope.std(axis=1), color='darkblue', alpha=0.3, lw=0)
ax.annotate('V1 - Allen', (1,0.05), color='darkblue', xycoords='axes fraction', ha='right')
ax.set_xticks([2,4,6,8,10])
ax.set_xlabel('freq. (Hz)')
ax.set_ylabel('LFP env. ($\mu$V)')
plt.tight_layout()
fig.savefig('../doc/delta-oscill-comparison.png')

## Find the optimal $\alpha$ parameter

this loop over session and $\alpha$ values is a bit long to run, ~15min

In [95]:
alpha_values = np.linspace(1.5, 7, 12)

output = np.zeros((len(sessions), len(alpha_values)))
for i in range(len(sessions)):
    #print('session', i)
    data = Data(session_index=i, reduced=True)
    data.compute_NSI(quantity='pop_act', alpha=2, verbose=False)
    for a, alpha in enumerate(alpha_values):
        data.compute_NSI(quantity='pLFP',
                         alpha=alpha,
                         p0_percentile=1,
                         verbose=False)
        data.validate_NSI(quantity='pLFP', verbose=False)
        pop_act_low_freq_env = resample_trace(data.t_pop_act, data.pop_act_low_freq_env, data.t_pLFP)
        delta_cond_validated = data.i_pLFP_vNSI & (data.pLFP_NSI<0)
        output[i, a] = np.mean(pop_act_low_freq_env[delta_cond_validated])

/home/yann.zerlaut/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/yann.zerlaut/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# plotting and fitting the decay
fig, ax = plt.subplots(1, figsize=(2.3,2.3))

# fitting an exponential
from scipy.optimize import curve_fit
def exp(t, decay,
        t0=alpha_values[0],
        x0=np.nanmean(output, axis=0)[-1],
        x1=np.nanmean(output, axis=0)[0]):
    return x0+(x1-x0)*np.exp(-(t-t0)/decay)
fit = curve_fit(exp, alpha_values, np.nanmean(output, axis=0), p0=10)
    
# plotting
ax.plot(alpha_values, np.nanmean(output, axis=0), color='darkblue')
ax.plot(alpha_values, exp(alpha_values, fit[0]), 'r:')
ax.annotate('$\\alpha_{top}$=%.2f' % (alpha_values[0]+fit[0]), (1,.7), ha='right', va='top', xycoords='axes fraction', color='r')
ax.fill_between(alpha_values, 
                 np.nanmean(output, axis=0)-np.nanstd(output, axis=0),
                 np.nanmean(output, axis=0)+np.nanstd(output, axis=0),color='darkblue', alpha=0.3, lw=0)
ax.annotate('n=11 sessions\nV1 - Allen', (1,1), ha='right', va='top', xycoords='axes fraction', color='darkblue')
ax.set_xlabel('alpha (unitless)')
ax.set_ylabel('rate $\delta$ env. (Hz)')
plt.tight_layout()
fig.savefig('../doc/alpha-delta-dep.png')

## Characterizing Network State Distributions

In [102]:
NSI_bins = np.linspace(-50, 80, 50)
NSI_hist = np.zeros((len(sessions), len(NSI_bins)-1))
for i in range(len(sessions)):
    data = Data(session_index=i, reduced=True)
    data.compute_NSI(quantity='pLFP',
                     alpha=3.11,
                     p0_percentile=1,
                    verbose=False)
    data.validate_NSI(quantity='pLFP', verbose=False)
    NSI_hist[i,:] = np.histogram(data.pLFP_vNSI, bins=NSI_bins, density=True)[0]
    

In [ ]:
fig, ax = plt.subplots(1, figsize=(2.3,2.3))
x = .5*(NSI_bins[1:]+NSI_bins[:-1])
for i in range(NSI_hist.shape[0]):
    ax.plot(x, NSI_hist[i,:], '-', lw=0.2, color='grey')
ax.annotate('run. disk\nV1 - Allen', (0,1), xycoords='axes fraction', color='darkblue', va='top')
ax.annotate('n=11 sessions', (1,1), xycoords='axes fraction', color='darkblue', va='top', ha='right', rotation=90, size='small')
ax.plot(x, NSI_hist.mean(axis=0), '-', color='darkblue')
ax.fill_between(x, NSI_hist.mean(axis=0)-NSI_hist.std(axis=0), NSI_hist.mean(axis=0)+NSI_hist.std(axis=0), color='darkblue', alpha=.3)
ax.set_ylabel('norm. count')
ax.set_yticks([])
ax.set_xlabel('NSI$_{pLFP}$ ($\mu$V)')
ax.set_xticks([-30,0,30,60])
plt.tight_layout()
fig.savefig('../doc/V1-NSI-hist.png')

In [ ]:
minc=1e-4 # min count
fig, ax = plt.subplots(1, figsize=(2.3,2.3))
x = .5*(NSI_bins[1:]+NSI_bins[:-1])
for i in range(NSI_hist.shape[0]):
    ax.plot(x, NSI_hist[i,:]+minc, '-', lw=0.2, color='grey')
ax.annotate('run. disk\nV1 - Allen', (0,1), xycoords='axes fraction', color='darkblue', va='top')
ax.annotate('n=11 sessions', (1,1), xycoords='axes fraction', color='darkblue', va='top', ha='right', rotation=90, size='small')
mean = NSI_hist.mean(axis=0)
mean[mean<minc] = minc
ax.plot(x, mean, '-', color='darkblue')
lb, hb = NSI_hist.mean(axis=0)-NSI_hist.std(axis=0), NSI_hist.mean(axis=0)+NSI_hist.std(axis=0)
lb[lb<minc]=minc
hb[hb<minc]=minc
ax.fill_between(x, lb, hb, color='darkblue', alpha=.3)
ax.set_ylabel('norm. log count')
ax.set_yticks([])
ax.set_xlabel('NSI$_{pLFP}$ ($\mu$V)')
ax.set_xticks([-30,0,30,60])
plt.yscale('log')
plt.tight_layout()
fig.savefig('../doc/V1-NSI-log-hist.png')

In [34]:
# let's compare to S1, loading a reduced dataset per cell

NSI_bins_S1 = np.linspace(-15, 20, 50)

if os.path.isfile('reduced_data/reduced_data_cell_1.npz'):
    # if you have the S1 data, let's do the same on S1
    NSI_hist_S1 = np.zeros((14, len(NSI_bins_S1)-1))
    for i in np.arange(1, 15):
        data = dict(np.load('reduced_data/reduced_data_cell_'+str(i)+'.npz'))
        NSI_hist_S1[i-1,:] = np.histogram(data['NSI'][data['NSI_validated']], bins=NSI_bins_S1, density=True)[0]
    # plot
    fig, ax = plt.subplots(1, figsize=(2.3,2.3))
    x = .5*(NSI_bins_S1[1:]+NSI_bins_S1[:-1])
    for i in range(NSI_hist_S1.shape[0]):
        ax.plot(x, NSI_hist_S1[i,:], '-', lw=0.2, color='grey')

    ax.annotate('quietly-sitting\nS1', (0,1), xycoords='axes fraction', color='darkgreen', va='top')
    ax.annotate('n=14 rec', (1,1), xycoords='axes fraction', color='darkgreen', va='top', ha='right', rotation=90, size='small')
    ax.plot(x, NSI_hist_S1.mean(axis=0), lw=2, color='darkgreen')
    ax.fill_between(x, NSI_hist_S1.mean(axis=0)-NSI_hist_S1.std(axis=0), NSI_hist_S1.mean(axis=0)+NSI_hist_S1.std(axis=0), color='darkgreen', alpha=.3)
    ax.set_ylabel('norm. count')
    ax.set_xlabel('NSI$_{pLFP}$ ($\mu$V)')
    ax.set_yticks([])
    ax.set_xticks([-10,0,10,20])   
    plt.tight_layout()
    fig.savefig('../doc/S1-NSI-hist.png')
else:
    print(' S1 data not found')

## Studying the population rate correlate of NSI$_{pLFP}$-based network states

In [ ]:
alpha_values = np.linspace(1.5, 6, 10)

output = np.zeros((len(sessions), len(alpha_values)))
for i in range(len(sessions)):
    #print('session', i)
    data = Data(session_index=i, reduced=True)
    data.compute_NSI(quantity='pop_act', alpha=2, verbose=False)
    for a, alpha in enumerate(alpha_values):
        data.compute_NSI(quantity='pLFP',
                         alpha=alpha,
                         p0_percentile=1,
                         verbose=False)
        data.validate_NSI(quantity='pLFP', verbose=False)
        pop_act_low_freq_env = resample_trace(data.t_pop_act, data.pop_act_low_freq_env, data.t_pLFP)
        delta_cond_validated = data.i_pLFP_vNSI & (data.pLFP_NSI<0)
        output[i, a] = np.mean(pop_act_low_freq_env[delta_cond_validated])

## We plot sample data

In [1]:
def plot_sample_data(data, title='',
                     time_points=[100, 1000],
                     duration=2):


    fig, AX_full = plt.subplots(6,len(time_points), figsize=(2.3*len(time_points), 7))
    fig.suptitle(title)
    
    if len(time_points)==1:
        AX_full = [AX_full]
        
    YLIMS = [[np.inf, -np.inf] for i in range(len(AX_full))]
    for t0, AX in zip(time_points, AX_full.T):
        
        # raster plot
        if hasattr(data, 'V1_RASTER'):
            for i, spikes in enumerate(data.V1_RASTER):
                cond = (spikes>t0) & (spikes<(t0+duration))
                AX[0].plot(spikes[cond], i+0*spikes[cond], 'o', ms=0.4, color='darkblue')
                
        AX[0].set_ylabel('units')
        
        # pop act. plot
        data.plot('pop_act', t0=t0, duration=duration, ax=AX[1], color='darkblue')
        data.plot('pop_act_sliding_mean', t0=t0, duration=duration, ax=AX[1], color='darkblue', lw=5, alpha=.2)
        
        # LFP plot
        data.plot('LFP', t0=t0, duration=duration, ax=AX[2], color='dimgrey')
        
        # pLFP plot
        data.plot('pLFP', t0=t0, duration=duration, ax=AX[3], color=plt.cm.tab10(5))
        data.plot('pLFP_sliding_mean', t0=t0, duration=duration, ax=AX[3], color=plt.cm.tab10(5), lw=5, alpha=.2)

        # NSI plot
        data.plot('pLFP_NSI', t0=t0, duration=duration, ax=AX[4], color='k', lw=1)
        data.plot('pLFP_vNSI', t0=t0, duration=duration, ax=AX[4], color=plt.cm.tab10(5), lw=0, ms=5)

        # speed plot
        data.plot('running_speed', t0=t0, duration=duration, ax=AX[5])

        # labelling axes and setting the same limes
        for j, label, ax in zip(range(len(AX)), 
                                ['units', 'rate (Hz)', 'LFP (mV)', '  pLFP ($\mu$V)', 'NSI$_{pLFP}$ ($\mu$V)', 'run. speed\n (cm/s)'],
                                AX):
            if ax in AX_full.T[0]:
                ax.set_ylabel(label)
            ax.set_xticks([])
            ax.set_xlim([t0,t0+duration])
            YLIMS[j] = [np.min([AX[j].get_ylim()[0], YLIMS[j][0]]),
                        np.max([AX[j].get_ylim()[1], YLIMS[j][1]])]
        AX[0].set_title('    $t_0$=%.1fs' % (t0-data.t0), size='small')
        
    for AX in AX_full.T:
        for j in range(len(AX)):
            ylim = [YLIMS[j][0]-.05*(YLIMS[j][1]-YLIMS[j][0]),
                    YLIMS[j][1]+.05*(YLIMS[j][1]-YLIMS[j][0])]
            AX[j].set_ylim(ylim)

    for t0, AX in zip(time_points, AX_full.T):
        AX[0].plot([t0,t0+0.2], (YLIMS[0][1])*np.ones(2), 'k-', lw=1)
        AX[0].annotate('200ms', (0,1), xycoords='axes fraction')# (t0, YLIMS[0][1]))
    
    return fig, AX
        


In [151]:
data = Data(session_index=5, demo=False, reduced=True)
            
data.compute_NSI(quantity='pLFP',
                 alpha=3.2,
                 p0_percentile=1)
data.compute_NSI(quantity='pop_act',
                 alpha=2.,
                 p0_percentile=0)
data.validate_NSI(quantity='pLFP')

 - computing NSI for "pLFP" [...]
 - - > done !
 - computing NSI for "pop_act" [...]
 - - > done !
 - validating NSI for "pLFP" [...]
 - - > done !


In [147]:
def look_for_good_trials(data, nsi_plfp_level,
                         nsi_plfp_tolerance=3,
                         rate_tolerance=2,
                         with_running=False, 
                         run_thresh=3):
    
    pop_act_NSI_resampled = resample_trace(data.t_pop_act, 
                                           data.pop_act_NSI, data.t_pLFP)
    

    x, y = data.pLFP_NSI[data.i_pLFP_vNSI], pop_act_NSI_resampled[data.i_pLFP_vNSI]
    cond = ((x>0) & (y>0)) | ((x<0) & (y<0))

    lin = np.polyfit(x[cond], y[cond], 1)
    
    accuracy_cond = (np.abs(y-np.polyval(lin, x))<rate_tolerance) & \
        (np.abs(x-nsi_plfp_level)<nsi_plfp_tolerance)
    
    if with_running:
        times = []
        for t in data.t_pLFP[data.i_pLFP_vNSI][accuracy_cond]:
            if data.t_running_speed[np.argmin((t-data.t_running_speed)**2)]>run_thresh:
                times.append(t)
        return times
    else:
        return data.t_pLFP[data.i_pLFP_vNSI][accuracy_cond]



array([7026.89042951, 7027.08962955, 7040.23683261, 7055.77443622,
       7094.61844526, 7095.01684535, 7813.73061247, 7900.38263262,
       7945.40184309])

In [ ]:
data = Data(session_index=1, demo=False, reduced=True)
            
data.compute_NSI(quantity='pLFP',
                 alpha=3.2,
                 p0_percentile=1)
data.compute_NSI(quantity='pop_act',
                 alpha=1.8,
                 p0_percentile=0)
data.validate_NSI(quantity='pLFP')

get_accuracy(data)

In [148]:
look_for_good_trials(data, -25, 2)

array([6956.37361311, 6956.57281316, 6967.13041561, 7052.18883539,
       7056.17283632, 7094.41924521, 7095.61444549, 7542.02174929,
       7683.85218227, 7684.05138232, 7815.32421284])

In [ ]:
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d 

def get_accuracy(data,
                 rate_tolerance=3,
                 with_fig=True):
    
    pop_act_NSI_resampled = resample_trace(data.t_pop_act, 
                                           data.pop_act_NSI, data.t_pLFP)
    

    x, y = data.pLFP_NSI[data.i_pLFP_vNSI], pop_act_NSI_resampled[data.i_pLFP_vNSI]
    cond = ((x>0) & (y>0)) | ((x<0) & (y<0))

    lin = np.polyfit(x[cond], y[cond], 1)
    
    accuracy_cond = np.abs(y-np.polyval(lin, x))<rate_tolerance
    
    accuracy = 100*np.sum(accuracy_cond)/len(y)    
    if with_fig:
        fig, ax = plt.subplots(1, figsize=(2,2))
        ax.set_title('accuracy=%.1f%%'%accuracy, fontsize=11)
        x = np.linspace(x.min(), x.max())
        ax.plot(data.pLFP_NSI[data.i_pLFP_vNSI], 
                 pop_act_NSI_resampled[data.i_pLFP_vNSI], 'o', lw=1, ms=0.2)
        ax.fill_between(x, np.polyval(lin, x)-rate_tolerance, np.polyval(lin, x)+rate_tolerance, color='g', alpha=.3)
        ax.plot(x, np.polyval(lin, x), 'k-')
        ax.set_ylabel('NSI$_{\,rate}$ (Hz)')
        ax.set_xlabel('NSI$_{\,pLFP}$ ($\mu$V)')
        return fig, ax, accuracy
    else:
        return accuracy
    
get_accuracy(data)

In [22]:
#%run exp_data.py
# loading a reduced dataset per cell
DATA = []
for i in np.arange(1, 15):
    data = dict(np.load('reduced_data/reduced_data_cell_'+str(i)+'.npz'))
    print(data['sbsmpl_Extra'][0], data['dt'])


0.059509277 5e-05
0.078430176 5e-05
0.14312744 4.999999998744897e-05
0.18432617 4.999999998744897e-05
0.14404297 4.999999998744897e-05
0.06652832 5.000000000165983e-05
0.066833496 5.000000000165983e-05
0.18981934 5.000000000165983e-05
0.3338623 5.000000000165983e-05
0.2154541 5e-05
-0.25115967 5e-05
-0.025024414 5e-05
-0.021057129 5e-05
0.30670166 5e-05


In [ ]:
OUTPUT = dict(np.load('cortical_arousal_index/data/final_alpha.npz'))
fig_lf, ax2 = figure(figsize=(.75, 1.), top=1.5)
mean, std, alpha = [], [], []
for i in range(OUTPUT['VM_LOW_FREQ_POWER'].shape[0])[3::2]:
    cond = np.isfinite(OUTPUT['VM_LOW_FREQ_POWER'][i,:]) & (OUTPUT['N_LOW_FREQ'][i,:]>10)
    mean.append(np.mean(OUTPUT['VM_LOW_FREQ_POWER'][i,:][cond]))
    std.append(np.std(OUTPUT['VM_LOW_FREQ_POWER'][i,:][cond]))
    alpha.append(OUTPUT['ALPHA'][i])
ylim = [4.1,10.5]
color=Purple    
ax2.plot(alpha, np.array(mean), color=Purple, lw=3)
ax2.fill_between(alpha, np.array(mean)+np.array(std),
                 np.array(mean)-np.array(std), color=Purple, alpha=.3, lw=0)
# exponential fit
def exp(x, p):
    return p[2]+p[1]*np.exp(-np.array(x)/p[0])
def to_minimize(p):
    return np.sum(np.abs(mean-exp(np.array(alpha)-alpha[0], p)))
from scipy.optimize import minimize
plsq = minimize(to_minimize, [2., 5., 5.])
ax2.plot(np.ones(2)*(alpha[0]+plsq.x[0]), [ylim[0], ylim[1]-1], '--', color=Brown, lw=.5)
ax2.annotate('$\\alpha^{opt}$', (alpha[0]+plsq.x[0], ylim[1]-1), color=Brown, fontsize=FONTSIZE)
ax2.plot((alpha[0], alpha[0]+plsq.x[0], alpha[-1]), [mean[0], plsq.x[2], plsq.x[2]], '--', color=Brown, lw=.5)
ax2.plot(alpha, exp(alpha-alpha[0], plsq.x), '--', color='k', lw=1, label='exp. fit')
ax2.legend(loc=(-.1,1.05), frameon=False, prop={'size':FONTSIZE-1}, handlelength=1.8, handletextpad=0.5)
set_plot(ax2, xlabel='$\\alpha$ (unitless)', ylabel='$V_m$ $\delta_{env}$ (mV)   ', ylim=ylim)
alpha0 = alpha[0]+plsq.x[0]
print(round(alpha0,2))
update_study_output('alpha_opt', str(round(alpha0,2)))